In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import os
os.chdir('..')
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings
from langchain_core.messages import HumanMessage

In [15]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document

from langchain_chinese import (
    AskDocumentTool,
    create_qa_chain,
    create_reason_agent,
)
from langchain_chinese.langgraph import create_tools_calling_executor

query_document = Document(page_content="langchain_chinese 是为中国大语言模型优化的langchain模块")

db = Chroma.from_documents([query_document], ZhipuAIEmbeddings())

llm_zhipu = ChatZhipuAI(model="glm-4")

chain = create_qa_chain(llm_zhipu, db.as_retriever())
qa_tool = AskDocumentTool(qa_chain=(chain | StrOutputParser()), name="qa_chain")

In [27]:
chain.get_graph().print_ascii()

              +---------------------------------+          
              | Parallel<context,question>Input |          
              +---------------------------------+          
                    ****               ****                
                 ***                       ***             
               **                             ***          
    +-------------+                              **        
    | Lambda(...) |                               *        
    +-------------+                               *        
            *                                     *        
            *                                     *        
            *                                     *        
+----------------------+                          *        
| VectorStoreRetriever |                          *        
+----------------------+                          *        
            *                                     *        
            *                           

In [17]:
chain.invoke(["langchain_chinese 是啥"])

AIMessage(content='langchain_chinese 是专门针对中国大语言模型进行优化的langchain模块。这意味着它对langchain的功能进行了调整和改进，以更好地适应和满足中国大语言模型的特定需求。')

In [18]:
from langchain.tools import tool
import random

# @tool(args_schema=WhereIsCatSchema)
@tool("WhereIsCatHidding")
def where_is_cat_hiding(idea: str) -> str:
    """从这些地方选择猫躲藏的地方"""
    return random.choice(["在床底下", "在书架中", "在阳台"])

In [19]:
# from langchain_openai import ChatOpenAI
# llm_zhipu = ChatZhipuAI()
tools = [qa_tool, where_is_cat_hiding]
app = create_tools_calling_executor(llm_zhipu, tools)

In [20]:
from langchain_core.messages import HumanMessage
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    verbose = True)

inputs = [HumanMessage(content="请查询资料，告诉我langchain_chinese是什么？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__runnable-name:  qa_chain
log: call tool [qa_chain]
{}
{'chunk': [ToolMessage(content='根据您提供的资料，"langchain_chinese" 是专门针对中国大语言模型进行优化的 langchain 模块。这个模块的设计目的很可能是为了更好地适应中文语言环境下的需求，提供更加精准和高效的语言处理能力。至于您的问题，似乎不完整，如果“l”是您想询问的内容的话，请您提供更详细的问题，我会依据所给资料为您解答。', tool_call_id='call_8483741594421818220')]}
{'input': [HumanMessage(content='请查询资料，告诉我langchain_chinese是什么？'), AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8483741594421818220', 'function': {'arguments': '{"query":"langchain_chinese是什么？"}', 'name': 'qa_chain'}, 'type': 'function'}]})], 'output': [ToolMessage(content='根据您提供的资料，"langchain_chinese" 是专门针对中国大语言模型进行优化的 langchain 模块。这个模块的设计目的很可能是为了更好地适应中文语言环境下的需求，提供更加精准和高效的语言处理能力。至于您的问题，似乎不完整，如果“l”是您想询问的内容的话，请您提供更详细的问题，我会依据所给资料为您解答。', tool_call_id='call_8483741594421818220')]}
经过_查询_，_"_lang_chain__ch_inese_"_ 是_专门_针对_中国_大_语言_模型_进行_优_化的_ lang_chain_ 模_块_。_这个_模块_的设计_目的_很_可能是_为了_更好地_适应_中文_语言_环境_下的_需求_，_提供_更加_精准_和_高效_的语言_处理_能力_。_如果您_有_关于_ lang_chain__ch_inese

In [21]:
app.get_graph().print_ascii()

                    +-----------+             
                    | __start__ |             
                    +-----------+             
                          *                   
                          *                   
                          *                   
                      +-------+               
                      | agent |               
                     *+-------+*              
                   **           ***           
                 **                **         
               **                    **       
+-----------------------+              **     
| agent_should_continue |               *     
+-----------------------+               *     
            *           *****           *     
            *                ****       *     
            *                    ***    *     
       +---------+                +--------+  
       | __end__ |                | action |  
       +---------+                +--------+  


In [22]:
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    runnables = {"qa_chain": chain},
    verbose = True)

inputs = ["请查询资料，告诉我langchain_chinese是什么？"]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__runnable-name:  qa_chain
log: call runnable [qa_chain]
lang_chain__ch_inese_ 是_专门_为中国_的大_语言_模型_优_化的_ lang_chain_ 模_块_。_这意味着_它_对_ lang_chain_ 进行_了_调整_和_改进_，_以便_更好地_适应_和_服务于_中国_的大_语言_模型_应用_场景_。__

In [23]:
app.get_graph().print_ascii()

                        +-----------+                     
                        | __start__ |                     
                        +-----------+                     
                              *                           
                              *                           
                              *                           
                          +-------+                       
                          | agent |*                      
                         *+-------+ ****                  
                       **               ***               
                     **                    ***            
                   **                         ****        
    +-----------------------+                     **      
    | agent_should_continue |                      *      
    +-----------------------+**                    *      
           **        **        *******             *      
         **            **             *****        *    

In [24]:
inputs = [HumanMessage(content="猫在哪里？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__runnable-name:  WhereIsCatHidding
log: call tool [WhereIsCatHidding]
{}
{'chunk': [ToolMessage(content='在阳台', tool_call_id='call_8483746542224243147')]}
{'input': [HumanMessage(content='猫在哪里？'), AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8483746542224243147', 'function': {'arguments': '{"idea":"猫在哪里"}', 'name': 'WhereIsCatHidding'}, 'type': 'function'}]})], 'output': [ToolMessage(content='在阳台', tool_call_id='call_8483746542224243147')]}
猫_在_阳_台上_躲_藏着_呢_！__

In [25]:
inputs =  [HumanMessage(content="霍金的生日是哪一天？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__runnable-name:  qa_chain
log: call runnable [qa_chain]


In [26]:
inputs =  [HumanMessage(content="写一个关于程序员的笑话")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

为什么_程序员_总是_携带_电脑_？_因为他们_不想_被人_称为_“_裸_奔_者_”。__